In [ ]:
%%capture
!pip install --no-cache-dir "unsloth==2025.11.2"
!pip install accelerate bitsandbytes huggingface_hub transformers neo4j_graphrag neo4j func-timeout

**Nhập API The Hugging Face để sử dụng**

In [ ]:
from huggingface_hub import login
login()

**Import thư viện**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from neo4j_graphrag.schema import get_structured_schema
from func_timeout import func_timeout, FunctionTimedOut
from neo4j.exceptions import AuthError, Neo4jError
from neo4j import GraphDatabase
from datetime import datetime
import pandas as pd
import torch
import time
import json
import csv
import os
import re

In [ ]:
import logging
import warnings
logging.getLogger('neo4j').setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=FutureWarning)

**Load model**

In [ ]:
model_name = "qwen" # Chọn "qwen" hoặc "llama3"
model_id = "unsloth/Qwen2.5-7B-Instruct"
max_seq_length = 4096
dtype = None
load_in_4bit = True  # Dùng 4bit để tiết kiệm VRAM

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Chế độ Inference (Tăng tốc độ lên 2x)
FastLanguageModel.for_inference(model)

# Setup chat template
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
    map_eos_token = True,
)

==((====))==  Unsloth 2025.11.2: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

**Load data**

In [ ]:
checkpoint_path = '/content/drive/MyDrive/T2C_qwen_bs_schema/qwen_bs_schema.csv'
test_path = '/content/drive/MyDrive/T2C_qwen_bs_schema/text2cypher_test.csv'

test_df = pd.read_csv(test_path)
print(f"Loaded test shape: {test_df.shape}")

Loaded test shape: (4833, 6)


**Prompt Text2Cypher (1)**

In [ ]:
def prompt_T2C(question, schema):
    system_message = """Let's think step by step.
    Task: Generate a Cypher statement to query a graph database. Instructions: Use only the provided relationship types and properties in the schema. Do not use any other relationship types or properties that are not provided in the schema. Do not include any explanations or apologies in your responses. Do not respond to any questions that ask anything other than constructing a Cypher statement. Do not include any text except the generated Cypher statement."""

    user_content = f"""Generate Cypher statement to query a graph database. Use only the provided relationship types and properties in the schema.
 Schema: {schema}
 Question: {question}
 Cypher output:"""

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content}
    ]

    return messages

print("prompt_T2C loaded successfully")

prompt_T2C loaded successfully


**Generate cypher**

In [ ]:
def generate_cypher_raw(question, schema):
    messages = prompt_T2C(question, schema)
    try:
        # Apply chat template và tokenize
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to(model.device)
        # Create attention mask
        attention_mask = (inputs != tokenizer.pad_token_id).long()
        # Generate
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs,
                attention_mask=attention_mask,
                max_new_tokens=256,
                temperature=0.1,
                do_sample=True,
                top_p=0.9,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
            )

        # Decode output
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)

        return generated_text

    except Exception as e:
        return "error"

print("generate_cypher_raw loaded successfully")

generate_cypher_raw loaded successfully


**Format query từ output**

In [ ]:
MODEL_TOKENS = {
    "qwen": {"start": "<|im_start|>assistant", "end": "<|im_end|>", "special_tokens": ["<|im_end|>", "<|im_start|>", "<|eot_id|>", "<|start_header_id|>", "<|end_header_id|>", "<|endoftext|>"]},
    "llama3": {"start": "<|start_header_id|>assistant<|end_header_id|>", "end": "<|eot_id|>", "special_tokens": ["<s>", "</s>", "<pad>"]}
}

def extract_cypher(text, model_name):
    if text in ["time_error", "error"]:
        return text

    try:
        # Bước 1: Extract assistant content
        tokens = MODEL_TOKENS[model_name]
        if tokens["start"] in text:
            after_assistant = text.split(tokens["start"])[-1]
            if tokens["end"] in after_assistant:
                assistant_content = after_assistant.split(tokens["end"])[0].strip()
            else:
                assistant_content = after_assistant.strip()
        else:
            assistant_content = text.strip()

        # Bước 2: Extract code block đầu tiên (giữa ``` ```)
        match = re.search(r"```(.*?)```", assistant_content, re.DOTALL)
        if match:
            code_block = match.group(1).strip()
        else:
            code_block = assistant_content.strip()

        # Bước 3: Tìm "MATCH" và trích xuất Cypher từ đó
        code_block_lower = code_block.lower()
        match_index = code_block_lower.find("match")
        if match_index == -1:
            return "error"

        cypher = code_block[match_index:].strip()

        # Loại bỏ special tokens
        for token in tokens["special_tokens"]:
            cypher = cypher.replace(token, "")
        cypher = cypher.strip()

        return cypher
    except:
        return "error"

print("✓ extract_cypher loaded")

✓ extract_cypher loaded


In [ ]:
def generate_cypher(question, schema, model_name=model_name):
    # Generate raw output
    raw_output = generate_cypher_raw(question, schema)

    # Extract Cypher
    cypher = extract_cypher(raw_output, model_name)
    return cypher

print("generate_cypher loaded successfully")

generate_cypher loaded successfully


**Load database sẽ sử dụng**

In [ ]:
URI = "neo4j+s://demo.neo4jlabs.com:7687"

# Danh sách alias
unique_aliases = test_df["database_reference_alias"].dropna().unique().tolist()
DATABASE_ALIASES = unique_aliases

# Lưu trữ drivers, schemas, examples theo alias
DRIVERS_BY_ALIAS = {}
SCHEMAS_BY_ALIAS = {}
EXAMPLES_BY_ALIAS = {}

**Helper driver**

In [ ]:
def extract_alias(alias: str):
    """Extract username và password từ alias"""
    name = alias.replace("neo4jlabs_demo_db_", "")
    return name, name

def get_driver(alias):
    """Lấy driver đã tồn tại hoặc tạo mới"""
    if alias in DRIVERS_BY_ALIAS:
        return DRIVERS_BY_ALIAS[alias]
    user, pwd = extract_alias(alias)
    driver = GraphDatabase.driver(URI, auth=(user, pwd))
    DRIVERS_BY_ALIAS[alias] = driver
    return driver

def reset_driver(alias):
    """Reset driver khi gặp lỗi auth"""
    print(f"Resetting driver for alias: {alias}")
    if alias in DRIVERS_BY_ALIAS:
        try:
            DRIVERS_BY_ALIAS[alias].close()
        except Exception as e:
            print(f"Error closing driver: {e}")
        del DRIVERS_BY_ALIAS[alias]

    # Tạo lại driver mới
    return get_driver(alias)

**Helper lấy example**

In [ ]:
def safe_ident(name):
    return f"`{name}`"

def infer_type(value):
    if value is None:
        return "STRING"
    if isinstance(value, bool):
        return "BOOL"
    if isinstance(value, int):
        return "INT"
    if isinstance(value, float):
        return "FLOAT"
    return "STRING"

def is_valid_example(value, max_length=15):
    if value is None:
        return False

    # Convert sang string để check độ dài
    value_str = str(value)

    # Check độ dài NGAY LẬP TỨC
    if len(value_str) > max_length:
        return False

    if isinstance(value, str):
        val_lower = value.lower()
        if val_lower == "null":
            return False

        # Chuỗi hex dài (check này bây giờ redundant vì đã check len)
        if re.fullmatch(r"[0-9a-fA-F]+", value) and len(value) > 30:
            return False

        # Base64 dài (check này cũng redundant)
        if re.fullmatch(r"[0-9A-Za-z+/=]+", value) and len(value) > 40:
            return False

    return True

def get_sample(tx, label, prop_name, limit=1):
    label_safe = safe_ident(label)
    prop_safe = safe_ident(prop_name)

    q = (
        f"MATCH (n:{label_safe}) "
        f"WHERE n.{prop_safe} IS NOT NULL "
        f"RETURN n.{prop_safe} AS value LIMIT {limit}"
    )
    res = tx.run(q)
    return [r["value"] for r in res]

def get_relationship_sample(tx, rel_type, prop_name, limit=1):
    rel_safe = safe_ident(rel_type)
    prop_safe = safe_ident(prop_name)

    q = (
        f"MATCH ()-[r:{rel_safe}]->() "
        f"WHERE r.{prop_safe} IS NOT NULL "
        f"RETURN r.{prop_safe} AS value LIMIT {limit}"
    )
    res = tx.run(q)
    return [r["value"] for r in res]

def find_mentioned_nodes(query_text, all_node_labels):
    mentioned = set()
    query_lower = query_text.lower()

    for label in all_node_labels:
        # Hỗ trợ cả label có ký tự đặc biệt
        pattern = r'\b' + re.escape(label.lower()) + r'\b'
        if re.search(pattern, query_lower):
            mentioned.add(label)

    return mentioned

**Hàm tạo sẵn schema kèm example**

In [ ]:
def example_alias(alias):
    driver = get_driver(alias)

    # Lấy schema
    try:
        schema = get_structured_schema(driver, is_enhanced=False)
    except AuthError as e:
        print(f"AuthError when getting schema for {alias}: {e}")
        driver = reset_driver(alias)
        schema = get_structured_schema(driver, is_enhanced=False)

    SCHEMAS_BY_ALIAS[alias] = schema

    node_props = schema.get("node_props", {})
    rel_props = schema.get("rel_props", {})
    examples = {"nodes": {}, "rels": {}}

    # Lấy ví dụ cho nodes
    with driver.session() as sess:
        for label, props in node_props.items():
            ex_node_props = {}
            for p in props:
                prop_name = p.get("property")
                if not prop_name:
                    continue
                try:
                    vals = sess.execute_read(get_sample, label, prop_name, 1)
                except AuthError as e:
                    print(f"AuthError sampling node {label}.{prop_name} for {alias}: {e}")
                    driver = reset_driver(alias)
                    with driver.session() as sess2:
                        vals = sess2.execute_read(get_sample, label, prop_name, 1)
                example = vals[0] if vals else None
                ex_node_props[prop_name] = example if is_valid_example(example) else None
            examples["nodes"][label] = ex_node_props

    # Lấy ví dụ cho relationships
    with driver.session() as sess:
        for rel_type, props in rel_props.items():
            ex_rel_props = {}
            for p in props:
                prop_name = p.get("property")
                if not prop_name:
                    continue
                try:
                    vals = sess.execute_read(get_relationship_sample, rel_type, prop_name, 1)
                except AuthError as e:
                    print(f"AuthError sampling rel {rel_type}.{prop_name} for {alias}: {e}")
                    driver = reset_driver(alias)
                    with driver.session() as sess2:
                        vals = sess2.execute_read(get_relationship_sample, rel_type, prop_name, 1)
                example = vals[0] if vals else None
                ex_rel_props[prop_name] = example if is_valid_example(example) else None
            examples["rels"][rel_type] = ex_rel_props

    EXAMPLES_BY_ALIAS[alias] = examples
    return SCHEMAS_BY_ALIAS[alias], EXAMPLES_BY_ALIAS[alias]

In [ ]:
for alias in DATABASE_ALIASES:
    try:
        example_alias(alias)
    except Exception as e:
        print(f"Failed to precompute {alias}: {e}")

**Format schema sang json**

In [ ]:
def convert_schema_json_format(schema, precomputed_examples, alias, node_labels_to_include=None):
    driver = get_driver(alias)

    if node_labels_to_include is None:
        node_labels_to_include = list(schema.get("node_props", {}).keys())

    unified_schema = {
        "nodes": {},
        "relationships": []
    }

    ex_nodes = precomputed_examples.get("nodes", {}) if precomputed_examples else {}

    # Convert nodes
    with driver.session() as sess:
        for label in node_labels_to_include:
            props = schema.get("node_props", {}).get(label, [])
            node_props = []

            for p in props:
                prop_name = p.get("property")
                if not prop_name:
                    continue

                # Ưu tiên dùng example đã precompute
                example = None
                if label in ex_nodes and prop_name in ex_nodes[label]:
                    example = ex_nodes[label][prop_name]

                # Nếu không có example sẵn -> truy vấn on-demand
                if example is None:
                    try:
                        vals = sess.execute_read(get_sample, label, prop_name, 1)
                    except AuthError as e:
                        print(f"AuthError in convert_schema (node) for {alias}: {e}")
                        driver = reset_driver(alias)
                        with driver.session() as sess2:
                            vals = sess2.execute_read(get_sample, label, prop_name, 1)
                    example = vals[0] if vals else None
                    if not is_valid_example(example):
                        example = None

                example_str = str(example) if example is not None else None
                dtype = infer_type(example) if example_str else "STRING"

                node_props.append({
                    "property": prop_name,
                    "type": dtype,
                    "example": example_str
                })

            unified_schema["nodes"][label] = node_props

    # Convert relationships
    ex_rels = precomputed_examples.get("rels", {}) if precomputed_examples else {}

    with driver.session() as sess:
        for rel_info in schema.get("relationships", []):
            rel_type = rel_info.get("type")
            start_label = rel_info.get("start")
            end_label = rel_info.get("end")

            if start_label in node_labels_to_include and end_label in node_labels_to_include:
                rel_props = []

                rel_prop_list = schema.get("rel_props", {}).get(rel_type, [])
                for p in rel_prop_list:
                    prop_name = p.get("property")
                    if not prop_name:
                        continue

                    example = None
                    if rel_type in ex_rels and prop_name in ex_rels[rel_type]:
                        example = ex_rels[rel_type][prop_name]

                    if example is None:
                        try:
                            vals = sess.execute_read(get_relationship_sample, rel_type, prop_name, 1)
                        except AuthError as e:
                            print(f"AuthError in convert_schema (rel) for {alias}: {e}")
                            driver = reset_driver(alias)
                            with driver.session() as sess2:
                                vals = sess2.execute_read(get_relationship_sample, rel_type, prop_name, 1)
                        example = vals[0] if vals else None
                        if not is_valid_example(example):
                            example = None

                    example_str = str(example) if example is not None else None
                    dtype = infer_type(example) if example_str else "STRING"

                    rel_props.append({
                        "property": prop_name,
                        "type": dtype,
                        "example": example_str
                    })

                unified_schema["relationships"].append({
                    "start": start_label,
                    "type": rel_type,
                    "end": end_label,
                    "properties": rel_props
                })

    return unified_schema

**Hàm schema linking**

In [ ]:
def filter_schema_by_query(query_text, alias):
    if alias not in SCHEMAS_BY_ALIAS:
        raise ValueError(f"Schema not found for alias: {alias}")

    schema = SCHEMAS_BY_ALIAS[alias]
    precomputed_examples = EXAMPLES_BY_ALIAS.get(alias)

    all_node_labels = list(schema.get("node_props", {}).keys())

    # Nếu schema có ít hơn hoặc bằng 3 nodes -> trả về full schema
    if len(all_node_labels) <= 3:
        return convert_schema_json_format(schema, precomputed_examples, alias, None)

    # Tìm mentioned nodes
    mentioned_nodes = find_mentioned_nodes(query_text, all_node_labels)

    # Không tìm thấy mentioned nodes -> trả về full schema
    if not mentioned_nodes:
        return convert_schema_json_format(schema, precomputed_examples, alias, None)

    # Có mentioned nodes và schema lớn -> filter
    return convert_schema_json_format(schema, precomputed_examples, alias, mentioned_nodes)

**Format schema sang markdow**

In [ ]:
def convert_schema_markdown_format(schema_dict):
    if not schema_dict:
        return None

    md_output = []
    md_output.append("### Nodes")

    # Format nodes
    for label, props in schema_dict.get("nodes", {}).items():
        md_output.append(f"- **{label}**")

        for prop in props:
            prop_name = prop["property"]
            dtype = prop["type"]
            example = prop.get("example")

            if example:
                md_output.append(f"  - `{prop_name}`: {dtype} Example: \"{example}\"")
            else:
                md_output.append(f"  - `{prop_name}`: {dtype}")

    # Format relationships
    md_output.append("\n### Relationships")

    relationships = schema_dict.get("relationships", [])
    if not relationships:
        md_output.append("- No relationships found")
    else:
        for rel in relationships:
            start = rel["start"]
            rel_type = rel["type"]
            end = rel["end"]
            rel_props = rel.get("properties", [])

            md_output.append(f"- **({start})-[:{rel_type}]->({end})**")

            for prop in rel_props:
                prop_name = prop["property"]
                dtype = prop["type"]
                example = prop.get("example")

                if example:
                    md_output.append(f"  - `{prop_name}`: {dtype} Example: \"{example}\"")
                else:
                    md_output.append(f"  - `{prop_name}`: {dtype}")

    return "\n".join(md_output).strip()

**Full schema format**

In [ ]:
def get_full_schema_formatted(alias):

    if alias not in SCHEMAS_BY_ALIAS:
        raise ValueError(f"Schema not found for alias: {alias}")

    schema = SCHEMAS_BY_ALIAS[alias]
    precomputed_examples = EXAMPLES_BY_ALIAS.get(alias)

    # Convert toàn bộ schema sang unified format
    unified = convert_schema_json_format(schema, precomputed_examples, alias, None)

    # Format sang markdown
    return convert_schema_markdown_format(unified)

**Chạy schema linking 1 dòng**

In [ ]:
def one_row_filter(query_text, alias):
    try:
        # Bước 1: Filter schema
        filtered_schema = filter_schema_by_query(query_text, alias)
        if not filtered_schema:
            return None

        # Bước 2: Format sang markdown
        formatted_schema = convert_schema_markdown_format(filtered_schema)
        return formatted_schema

    except Exception as e:
        print(f"Error processing {alias}: {e}")
        return None

In [ ]:
def generate_cypher2(question, alias, schema=None, timeout=300):
    # Kiểm tra alias có giá trị hay không
    if pd.isna(alias) or alias is None or str(alias).strip() == '':
        # Chỉ chạy bước 1, không có schema linking
        if schema is None:
            return ("error", schema)

        cypher_1 = generate_cypher(question, schema)
        return (cypher_1, schema)

    def _generate():
        schema_full = get_full_schema_formatted(alias)

        # BƯỚC 1: Generate Cypher lần 1
        cypher_1 = generate_cypher(question, schema_full)
        if cypher_1 in ["error"]:
            return (cypher_1, None)

        # BƯỚC 2: Extract schema linking từ Cypher(1) - CHỈ LÀM 1 LẦN
        schema_linked = one_row_filter(cypher_1, alias)
        if schema_linked is None:
            # Fallback về full schema nếu linking fail
            schema_linked = schema_full

        # BƯỚC 3: Generate Cypher lần 2 với schema đã link
        cypher_2 = generate_cypher(question, schema_linked)
        if cypher_2 in ["error"]:
            return (cypher_1, schema_linked)

        # Return cypher(2) và schema đã link
        return (cypher_2, schema_linked)

    try:
        return func_timeout(timeout, _generate)
    except FunctionTimedOut:
        return ("time_error", None)

In [ ]:
# Lấy test case đầu tiên
first_row = test_df.iloc[1]
test_question = first_row['question']
test_schema = first_row['schema']
test_alias = first_row['database_reference_alias']

print("="*80)
print("TEST QUESTION:")
print("="*80)
print(test_question)

# ============================================================================
# Test 1: Cypher
# ============================================================================
print("\n" + "="*80)
print("TEST 1: Cypher")
print("="*80)
result = generate_cypher2(test_question, test_alias, test_schema, 300)
print(result[0])

# ============================================================================
# Test 2: Expected Cypher
# ============================================================================
print("\n" + "="*80)
print("TEST 3: Expected Cypher")
print("="*80)
if 'cypher' in test_df.columns:
    print(first_row['cypher'])
else:
    print("N/A")

# ============================================================================
# Test 3: Sub Schema
# ============================================================================
print("\n" + "="*80)
print("TEST 3: Sub Schema")
print("="*80)
if result[1] is not None:
    print(result[1])
else:
    print("N/A")

TEST QUESTION:
What are the names of the technicians that have not been assigned to repair machines?

TEST 1: Cypher
MATCH (t:Technician)-[r:ASSIGNED_TO]->(ra:RepairAssignment)
OPTIONAL MATCH (ra)<-[:ASSIGNED_TO]-(m:Machine)
WHERE m IS NULL
RETURN t.Name

TEST 3: Expected Cypher
MATCH (t:Technician) WHERE NOT EXISTS ((:RepairAssignment)-[:ASSIGNED_TO]->(t)) RETURN t.Name

TEST 3: Sub Schema
{"ASSIGNED_TO": {"count": 27, "properties": {}, "type": "relationship"}, "Machine": {"count": 9, "labels": [], "properties": {"Making_Year": {"unique": false, "indexed": false, "type": "INTEGER", "existence": false}, "Machine_ID": {"unique": false, "indexed": false, "type": "INTEGER", "existence": false}, "value_points": {"unique": false, "indexed": false, "type": "FLOAT", "existence": false}, "Class": {"unique": false, "indexed": false, "type": "STRING", "existence": false}, "quality_rank": {"unique": false, "indexed": false, "type": "INTEGER", "existence": false}, "Team": {"unique": false, "indexe

In [ ]:
def run_with_checkpoint(
    test_df,
    checkpoint_path,
    timeout=300,
    log_interval=100
):
    # ==========================================================================
    # BƯỚC 1: Kiểm tra và load checkpoint
    if os.path.exists(checkpoint_path):
        print(f"[CHECKPOINT] Tìm thấy file checkpoint: {checkpoint_path}")
        df = pd.read_csv(checkpoint_path, encoding="utf-8-sig")
        print(f"[CHECKPOINT] Đã load {len(df)} dòng từ checkpoint")

        # Đếm số dòng đã xử lý
        processed_count = df['cypher_generated'].notna().sum()
        print(f"[CHECKPOINT] Đã xử lý: {processed_count}/{len(df)} dòng")

    else:
        print(f"[CHECKPOINT] Không tìm thấy checkpoint, tạo mới từ test_df")
        df = test_df.copy()
        df['cypher_generated'] = ''  # Thêm cột cypher_generated trống

        # Lưu file checkpoint ban đầu
        df.to_csv(checkpoint_path, index=False, encoding='utf-8')
        print(f"[CHECKPOINT] Đã tạo file checkpoint: {checkpoint_path}")

    # ==========================================================================
    # BƯỚC 2: Xử lý các dòng chưa có kết quả
    total_rows = len(df)

    # Biến đếm cho log
    success_count = 0
    error_count = 0
    timeout_error_count = 0
    batch_start_idx = 0

    # Biến đếm để lưu checkpoint
    processed_since_last_save = 0

    print(f"\n{'='*80}")
    print(f"BẮT ĐẦU XỬ LÝ - Tổng số dòng: {total_rows}")
    print(f"{'='*80}\n")

    start_time = time.time()

    for idx in range(total_rows):
        # Kiểm tra nếu dòng này đã có kết quả
        current_cypher = df.at[idx, 'cypher_generated']

        # Nếu đã có dữ liệu (không phải NaN và không phải chuỗi rỗng)
        if pd.notna(current_cypher) and str(current_cypher).strip() != '':
            continue  # SKIP dòng này

        # ======================================================================
        # XỬ LÝ DÒNG CHƯA CÓ KẾT QUẢ
        print(f"[Processing] Dòng {idx}...", end=" ", flush=True)

        try:
            question = df.at[idx, 'question']
            alias = df.at[idx, 'database_reference_alias']
            schema = df.at[idx, 'schema']

            # Gọi hàm generate_cypher2
            result = generate_cypher2(question, alias, schema, timeout)
            cypher_result = result[0]  # Lấy phần tử đầu tiên

            # Lưu kết quả
            df.at[idx, 'cypher_generated'] = cypher_result

            # Đếm theo loại kết quả và in log
            if cypher_result == "error":
                error_count += 1
                print("ERROR")
            elif cypher_result == "time_error":
                timeout_error_count += 1
                print("TIMEOUT")
            else:
                success_count += 1
                print("SUCCESS")

            # Tăng biến đếm để lưu checkpoint
            processed_since_last_save += 1

        except Exception as e:
            # Xử lý lỗi không mong đợi
            print(f"ERROR - {str(e)}")
            df.at[idx, 'cypher_generated'] = "error"
            error_count += 1
            processed_since_last_save += 1

        # ======================================================================
        # LƯU CHECKPOINT SAU MỖI 50 DÒNG
        # ======================================================================
        if processed_since_last_save >= 50:
            df.to_csv(checkpoint_path, index=False, encoding='utf-8')
            print(f"[CHECKPOINT] Đã lưu sau {processed_since_last_save} dòng")
            processed_since_last_save = 0

        # ======================================================================
        # LOG THỐNG KÊ MỖI 100 DÒNG
        # ======================================================================
        if (idx + 1) % log_interval == 0:
            elapsed_time = time.time() - start_time
            avg_time_per_row = elapsed_time / (idx + 1)
            remaining_rows = total_rows - (idx + 1)
            estimated_time = avg_time_per_row * remaining_rows

            print(f"\n{'='*80}")
            print(f"[LOG] Dòng {batch_start_idx}-{idx}")
            print(f"{'='*80}")
            print(f"Thành công:     {success_count}")
            print(f"Error:          {error_count}")
            print(f"Timeout Error:  {timeout_error_count}")
            print(f"Tổng xử lý:     {success_count + error_count + timeout_error_count}")
            print(f"Tiến độ:        {idx + 1}/{total_rows} ({(idx + 1)/total_rows*100:.2f}%)")
            print(f"Thời gian:      {elapsed_time/60:.2f} phút")
            print(f"Ước tính còn:   {estimated_time/60:.2f} phút")
            print(f"{'='*80}\n")

            # Reset bộ đếm cho batch tiếp theo
            success_count = 0
            error_count = 0
            timeout_error_count = 0
            batch_start_idx = idx + 1

    # ==========================================================================
    # LƯU CHECKPOINT CUỐI CÙNG (nếu còn dòng chưa lưu)
    # ==========================================================================
    if processed_since_last_save > 0:
        df.to_csv(checkpoint_path, index=False, encoding='utf-8')
        print(f"[CHECKPOINT] Đã lưu {processed_since_last_save} dòng cuối cùng")

    # ==========================================================================
    # KẾT THÚC - LOG CUỐI CÙNG
    # ==========================================================================
    total_time = time.time() - start_time

    # Đếm tổng kết quả
    final_success = (df['cypher_generated'].notna() &
                     (df['cypher_generated'] != 'error') &
                     (df['cypher_generated'] != 'time_error') &
                     (df['cypher_generated'] != '')).sum()
    final_error = (df['cypher_generated'] == 'error').sum()
    final_timeout = (df['cypher_generated'] == 'time_error').sum()

    print(f"\n{'='*80}")
    print(f"HOÀN THÀNH")
    print(f"{'='*80}")
    print(f"Tổng số dòng:        {total_rows}")
    print(f"Thành công:          {final_success} ({final_success/total_rows*100:.2f}%)")
    print(f"Error:               {final_error} ({final_error/total_rows*100:.2f}%)")
    print(f"Timeout Error:       {final_timeout} ({final_timeout/total_rows*100:.2f}%)")
    print(f"Tổng thời gian:      {total_time/60:.2f} phút")
    print(f"Thời gian trung bình: {total_time/total_rows:.2f} giây/dòng")
    print(f"Kết quả đã lưu tại:  {checkpoint_path}")
    print(f"{'='*80}\n")
    return df

In [ ]:
result_df = run_with_checkpoint(
    test_df=test_df,
    checkpoint_path=checkpoint_path,
    timeout=300,
    log_interval=100
)

[CHECKPOINT] Tìm thấy file checkpoint: /content/drive/MyDrive/T2C_qwen_bs_schema/qwen_bs_schema.csv
[CHECKPOINT] Đã load 4833 dòng từ checkpoint
[CHECKPOINT] Đã xử lý: 4450/4833 dòng

BẮT ĐẦU XỬ LÝ - Tổng số dòng: 4833

[Processing] Dòng 4450... SUCCESS
[Processing] Dòng 4451... SUCCESS
[Processing] Dòng 4452... SUCCESS
[Processing] Dòng 4453... SUCCESS
[Processing] Dòng 4454... SUCCESS
[Processing] Dòng 4455... SUCCESS
[Processing] Dòng 4456... SUCCESS
[Processing] Dòng 4457... SUCCESS
[Processing] Dòng 4458... SUCCESS
[Processing] Dòng 4459... SUCCESS
[Processing] Dòng 4460... SUCCESS
[Processing] Dòng 4461... SUCCESS
[Processing] Dòng 4462... SUCCESS
[Processing] Dòng 4463... SUCCESS
[Processing] Dòng 4464... SUCCESS
[Processing] Dòng 4465... SUCCESS
[Processing] Dòng 4466... SUCCESS
[Processing] Dòng 4467... SUCCESS
[Processing] Dòng 4468... SUCCESS
[Processing] Dòng 4469... SUCCESS
[Processing] Dòng 4470... SUCCESS
[Processing] Dòng 4471... SUCCESS
[Processing] Dòng 4472... SUCCES